# CAPSTONE

# Target = Outcome (1, 2 or 3)
- Multiclass Variable - 1 for Home team win, 2 for Away team win, 3 for Draw




## TO DO / METRICS TO CONSIDER

- DROP Unnecessary columns
- FILL in Nulls by extracting from EVENTS column

- POSESSION - is object. convert to numerical

- Current Form: Points earned in last 5 matches (max 15) rate on a 1-5 scale?


- Points per match: points / games played
- Shots on target per match: shots on target / total shots
- Current Form: Points earned in last 5 matches (max 15) rate on a 1-5 scale?

- Create opponent code? - will help when comparing head to head

- Opponent strength using form, table position, historical results
- How do teams compare head to head

In [13]:
#importing libraries
import pandas as pd
import numpy as np

In [14]:
#importing data set

dataset = pd.read_csv("full_data.csv", index_col=0)

In [15]:
dataset.head()

,League,Home,Away,INC,Round,Date,Time,H_Score,A_Score,HT_H_Score,...,A_Offsides,H_Throw_in,A_Throw_in,H_Goalkeeper_Saves,A_Goalkeeper_Saves,H_Fouls,A_Fouls,H_Yellow_Cards,A_Yellow_Cards,Game Link
0,championship,Swansea,Reading,"[""08' Yellow_Away - Griffin A."", ""12' Yellow_A...",Play-off,30.05.2011,16:00,4.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/feOBWmKD/#/ma...
1,championship,Cardiff,Reading,"[""28' Goal_Away - Long S."", ""44' Yellow_Home -...",Play-off,17.05.2011,20:45,0.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/IJUfyp3o/#/ma...
2,championship,Swansea,Nottingham,"[""26' Yellow_Away - Gunter C."", ""28' Goal_Home...",Play-off,16.05.2011,20:45,3.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/QoD4C0Qp/#/ma...
3,championship,Reading,Cardiff,"[""26' Yellow_Home - McAnuff J."", ""38' Yellow_H...",Play-off,13.05.2011,20:45,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/lMkyUSeI/#/ma...
4,championship,Nottingham,Swansea,"[""02' Red_Card_Away - Taylor N."", ""47' Yellow_...",Play-off,12.05.2011,20:45,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/6c0WU8tC/#/ma...


,League,Home,Away,INC,Round,Date,Time,H_Score,A_Score,HT_H_Score,...,A_Offsides,H_Throw_in,A_Throw_in,H_Goalkeeper_Saves,A_Goalkeeper_Saves,H_Fouls,A_Fouls,H_Yellow_Cards,A_Yellow_Cards,Game Link
0,championship,Swansea,Reading,"[""08' Yellow_Away - Griffin A."", ""12' Yellow_A...",Play-off,30.05.2011,16:00,4.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/feOBWmKD/#/ma...
1,championship,Cardiff,Reading,"[""28' Goal_Away - Long S."", ""44' Yellow_Home -...",Play-off,17.05.2011,20:45,0.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/IJUfyp3o/#/ma...
2,championship,Swansea,Nottingham,"[""26' Yellow_Away - Gunter C."", ""28' Goal_Home...",Play-off,16.05.2011,20:45,3.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/QoD4C0Qp/#/ma...
3,championship,Reading,Cardiff,"[""26' Yellow_Home - McAnuff J."", ""38' Yellow_H...",Play-off,13.05.2011,20:45,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/lMkyUSeI/#/ma...
4,championship,Nottingham,Swansea,"[""02' Red_Card_Away - Taylor N."", ""47' Yellow_...",Play-off,12.05.2011,20:45,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.flashscore.com/match/6c0WU8tC/#/ma...


In [16]:
dataset.League.value_counts()

premier-league        7600
laliga                7220
serie-a               7150
ligue-1               6757
championship          6684
league-one            6440
league-two            6015
bundesliga            5838
eredivisie            5776
laliga2               5519
serie-b               5286
ligue-2               4470
super-lig             3884
jupiler-league        3756
fortuna-1-liga        3687
2-bundesliga          3503
liga-portugal         3414
pko-bp-ekstraklasa    3338
Name: League, dtype: int64

premier-league        7600
laliga                7220
serie-a               7150
ligue-1               6757
championship          6684
league-one            6440
league-two            6015
bundesliga            5838
eredivisie            5776
laliga2               5519
serie-b               5286
ligue-2               4470
super-lig             3884
jupiler-league        3756
fortuna-1-liga        3687
2-bundesliga          3503
liga-portugal         3414
pko-bp-ekstraklasa    3338
Name: League, dtype: int64

In [17]:
dataset.dtypes

League                  object
Home                    object
Away                    object
INC                     object
Round                   object
Date                    object
Time                    object
H_Score                float64
A_Score                float64
HT_H_Score             float64
HT_A_Score             float64
WIN                     object
H_BET                  float64
X_BET                  float64
A_BET                  float64
WIN_BET                float64
OVER_2.5                object
OVER_3.5                object
H_15                      bool
A_15                      bool
H_45_50                   bool
A_45_50                   bool
H_90                      bool
A_90                      bool
H_Missing_Players      float64
A_Missing_Players      float64
Missing_Players        float64
H_Ball_Possession       object
A_Ball_Possession       object
H_Goal_Attempts        float64
A_Goal_Attempts        float64
H_Shots_on_Goal        float64
A_Shots_

League                  object
Home                    object
Away                    object
INC                     object
Round                   object
Date                    object
Time                    object
H_Score                float64
A_Score                float64
HT_H_Score             float64
HT_A_Score             float64
WIN                     object
H_BET                  float64
X_BET                  float64
A_BET                  float64
WIN_BET                float64
OVER_2.5                object
OVER_3.5                object
H_15                      bool
A_15                      bool
H_45_50                   bool
A_45_50                   bool
H_90                      bool
A_90                      bool
H_Missing_Players      float64
A_Missing_Players      float64
Missing_Players        float64
H_Ball_Possession       object
A_Ball_Possession       object
H_Goal_Attempts        float64
A_Goal_Attempts        float64
H_Shots_on_Goal        float64
A_Shots_

In [18]:
#convert data to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])

/var/folders/n4/dk0kn6092fg4rgl4rgty93tr0000gn/T/ipykernel_1846/3827951478.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dataset["Date"] = pd.to_datetime(dataset["Date"])
/var/folders/n4/dk0kn6092fg4rgl4rgty93tr0000gn/T/ipykernel_1846/3827951478.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dataset["Date"] = pd.to_datetime(dataset["Date"])


In [19]:
dataset.dtypes

League                         object
Home                           object
Away                           object
INC                            object
Round                          object
Date                   datetime64[ns]
Time                           object
H_Score                       float64
A_Score                       float64
HT_H_Score                    float64
HT_A_Score                    float64
WIN                            object
H_BET                         float64
X_BET                         float64
A_BET                         float64
WIN_BET                       float64
OVER_2.5                       object
OVER_3.5                       object
H_15                             bool
A_15                             bool
H_45_50                          bool
A_45_50                          bool
H_90                             bool
A_90                             bool
H_Missing_Players             float64
A_Missing_Players             float64
Missing_Play

League                         object
Home                           object
Away                           object
INC                            object
Round                          object
Date                   datetime64[ns]
Time                           object
H_Score                       float64
A_Score                       float64
HT_H_Score                    float64
HT_A_Score                    float64
WIN                            object
H_BET                         float64
X_BET                         float64
A_BET                         float64
WIN_BET                       float64
OVER_2.5                       object
OVER_3.5                       object
H_15                             bool
A_15                             bool
H_45_50                          bool
A_45_50                          bool
H_90                             bool
A_90                             bool
H_Missing_Players             float64
A_Missing_Players             float64
Missing_Play

In [20]:
print(f'The shape of the dataset is {dataset.shape[0]} rows and {dataset.shape[1]} columns')

The shape of the dataset is 96337 rows and 56 columns
The shape of the dataset is 96337 rows and 56 columns


In [21]:
print(dataset.columns)

Index(['League', 'Home', 'Away', 'INC', 'Round', 'Date', 'Time', 'H_Score',
       'A_Score', 'HT_H_Score', 'HT_A_Score', 'WIN', 'H_BET', 'X_BET', 'A_BET',
       'WIN_BET', 'OVER_2.5', 'OVER_3.5', 'H_15', 'A_15', 'H_45_50', 'A_45_50',
       'H_90', 'A_90', 'H_Missing_Players', 'A_Missing_Players',
       'Missing_Players', 'H_Ball_Possession', 'A_Ball_Possession',
       'H_Goal_Attempts', 'A_Goal_Attempts', 'H_Shots_on_Goal',
       'A_Shots_on_Goal', 'H_Attacks', 'A_Attacks', 'H_Dangerous_Attacks',
       'A_Dangerous_Attacks', 'H_Shots_off_Goal', 'A_Shots_off_Goal',
       'H_Blocked_Shots', 'A_Blocked_Shots', 'H_Free_Kicks', 'A_Free_Kicks',
       'H_Corner_Kicks', 'A_Corner_Kicks', 'H_Offsides', 'A_Offsides',
       'H_Throw_in', 'A_Throw_in', 'H_Goalkeeper_Saves', 'A_Goalkeeper_Saves',
       'H_Fouls', 'A_Fouls', 'H_Yellow_Cards', 'A_Yellow_Cards', 'Game Link'],
      dtype='object')
Index(['League', 'Home', 'Away', 'INC', 'Round', 'Date', 'Time', 'H_Score',
       'A_Score', 

In [22]:
dataset.tail()

,League,Home,Away,INC,Round,Date,Time,H_Score,A_Score,HT_H_Score,...,A_Offsides,H_Throw_in,A_Throw_in,H_Goalkeeper_Saves,A_Goalkeeper_Saves,H_Fouls,A_Fouls,H_Yellow_Cards,A_Yellow_Cards,Game Link
96332,league-two,Oldham,Newport,"[""14' Yellow_Away - Norman C."", ""14' Yellow_Ho...",1,2021-07-08,16:00,0.0,1.0,0.0,...,2.0,25.0,27.0,1.0,0.0,17.0,7.0,3.0,2.0,https://www.flashscore.com/match/CAtvrBOT/#/ma...
96333,league-two,Salford,Leyton Orient,"[""33' Goal_Away - Beckles O."", ""41' Goal_Home ...",1,2021-07-08,16:00,1.0,1.0,1.0,...,4.0,29.0,26.0,4.0,5.0,11.0,11.0,1.0,1.0,https://www.flashscore.com/match/IwAWkH61/#/ma...
96334,league-two,Scunthorpe,Swindon,"[""48' Goal_Home - Penalty Loft R.(Penalty )"", ...",1,2021-07-08,16:00,1.0,3.0,0.0,...,2.0,24.0,23.0,5.0,3.0,20.0,8.0,0.0,3.0,https://www.flashscore.com/match/WtEzkyM7/#/ma...
96335,league-two,Stevenage,Barrow,"[""21' Yellow_Away - Taylor J."", ""22' Yellow_Aw...",1,2021-07-08,16:00,1.0,0.0,0.0,...,4.0,26.0,26.0,2.0,2.0,20.0,14.0,4.0,6.0,https://www.flashscore.com/match/pp2IDKb8/#/ma...
96336,league-two,Tranmere,Walsall,"[""15' Yellow_Away - Taylor A."", ""42' Yellow_Ho...",1,2021-07-08,16:00,1.0,0.0,0.0,...,1.0,24.0,33.0,4.0,2.0,13.0,17.0,3.0,2.0,https://www.flashscore.com/match/Oz1MCvEE/#/ma...


,League,Home,Away,INC,Round,Date,Time,H_Score,A_Score,HT_H_Score,...,A_Offsides,H_Throw_in,A_Throw_in,H_Goalkeeper_Saves,A_Goalkeeper_Saves,H_Fouls,A_Fouls,H_Yellow_Cards,A_Yellow_Cards,Game Link
96332,league-two,Oldham,Newport,"[""14' Yellow_Away - Norman C."", ""14' Yellow_Ho...",1,2021-07-08,16:00,0.0,1.0,0.0,...,2.0,25.0,27.0,1.0,0.0,17.0,7.0,3.0,2.0,https://www.flashscore.com/match/CAtvrBOT/#/ma...
96333,league-two,Salford,Leyton Orient,"[""33' Goal_Away - Beckles O."", ""41' Goal_Home ...",1,2021-07-08,16:00,1.0,1.0,1.0,...,4.0,29.0,26.0,4.0,5.0,11.0,11.0,1.0,1.0,https://www.flashscore.com/match/IwAWkH61/#/ma...
96334,league-two,Scunthorpe,Swindon,"[""48' Goal_Home - Penalty Loft R.(Penalty )"", ...",1,2021-07-08,16:00,1.0,3.0,0.0,...,2.0,24.0,23.0,5.0,3.0,20.0,8.0,0.0,3.0,https://www.flashscore.com/match/WtEzkyM7/#/ma...
96335,league-two,Stevenage,Barrow,"[""21' Yellow_Away - Taylor J."", ""22' Yellow_Aw...",1,2021-07-08,16:00,1.0,0.0,0.0,...,4.0,26.0,26.0,2.0,2.0,20.0,14.0,4.0,6.0,https://www.flashscore.com/match/pp2IDKb8/#/ma...
96336,league-two,Tranmere,Walsall,"[""15' Yellow_Away - Taylor A."", ""42' Yellow_Ho...",1,2021-07-08,16:00,1.0,0.0,0.0,...,1.0,24.0,33.0,4.0,2.0,13.0,17.0,3.0,2.0,https://www.flashscore.com/match/Oz1MCvEE/#/ma...


In [25]:
#dropping unwanted columns

dataset = dataset.drop(["H_Missing_Players", "A_Missing_Players", "Missing_Players", "H_Offsides", "A_Offsides", "H_Throw_in", "A_Throw_in", "Round", "WIN_BET", "OVER_2.5", "OVER_3.5", "H_15",
                        "A_15",
                        "H_45_50", "A_45_50", "H_90", "A_90", "H_Attacks", "A_Attacks", "H_Dangerous_Attacks", "A_Dangerous_Attacks",
                       "H_Blocked_Shots", "A_Blocked_Shots", "H_Free_Kicks", "A_Free_Kicks", "H_Corner_Kicks", "A_Corner_Kicks",
                       "H_Goalkeeper_Saves", "A_Goalkeeper_Saves"], axis=1)

KeyError: "['H_Missing_Players', 'A_Missing_Players', 'Missing_Players', 'H_Offsides', 'A_Offsides', 'H_Throw_in', 'A_Throw_in', 'Round', 'WIN_BET', 'OVER_2.5', 'OVER_3.5', 'H_15', 'A_15', 'H_45_50', 'A_45_50', 'H_90', 'A_90', 'H_Attacks', 'A_Attacks', 'H_Dangerous_Attacks', 'A_Dangerous_Attacks', 'H_Blocked_Shots', 'A_Blocked_Shots', 'H_Free_Kicks', 'A_Free_Kicks', 'H_Corner_Kicks', 'A_Corner_Kicks', 'H_Goalkeeper_Saves', 'A_Goalkeeper_Saves'] not found in axis"

KeyError: "['H_Missing_Players', 'A_Missing_Players', 'Missing_Players', 'H_Offsides', 'A_Offsides', 'H_Throw_in', 'A_Throw_in', 'Round', 'WIN_BET', 'OVER_2.5', 'OVER_3.5', 'H_15', 'A_15', 'H_45_50', 'A_45_50', 'H_90', 'A_90', 'H_Attacks', 'A_Attacks', 'H_Dangerous_Attacks', 'A_Dangerous_Attacks', 'H_Blocked_Shots', 'A_Blocked_Shots', 'H_Free_Kicks', 'A_Free_Kicks', 'H_Corner_Kicks', 'A_Corner_Kicks', 'H_Goalkeeper_Saves', 'A_Goalkeeper_Saves'] not found in axis"

In [26]:
print(dataset.columns)

Index(['League', 'Home', 'Away', 'INC', 'Date', 'Time', 'H_Score', 'A_Score',
       'HT_H_Score', 'HT_A_Score', 'WIN', 'H_BET', 'X_BET', 'A_BET',
       'H_Ball_Possession', 'A_Ball_Possession', 'H_Goal_Attempts',
       'A_Goal_Attempts', 'H_Shots_on_Goal', 'A_Shots_on_Goal',
       'H_Shots_off_Goal', 'A_Shots_off_Goal', 'H_Fouls', 'A_Fouls',
       'H_Yellow_Cards', 'A_Yellow_Cards', 'Game Link'],
      dtype='object')
Index(['League', 'Home', 'Away', 'INC', 'Date', 'Time', 'H_Score', 'A_Score',
       'HT_H_Score', 'HT_A_Score', 'WIN', 'H_BET', 'X_BET', 'A_BET',
       'H_Ball_Possession', 'A_Ball_Possession', 'H_Goal_Attempts',
       'A_Goal_Attempts', 'H_Shots_on_Goal', 'A_Shots_on_Goal',
       'H_Shots_off_Goal', 'A_Shots_off_Goal', 'H_Fouls', 'A_Fouls',
       'H_Yellow_Cards', 'A_Yellow_Cards', 'Game Link'],
      dtype='object')


In [27]:
dataset.isna().sum()

League                   0
Home                     0
Away                     0
INC                     32
Date                     0
Time                     0
H_Score                 32
A_Score                 32
HT_H_Score              32
HT_A_Score              32
WIN                     32
H_BET                 1156
X_BET                 1156
A_BET                 1156
H_Ball_Possession    44450
A_Ball_Possession    44450
H_Goal_Attempts      43537
A_Goal_Attempts      43537
H_Shots_on_Goal      43540
A_Shots_on_Goal      43540
H_Shots_off_Goal     43999
A_Shots_off_Goal     43999
H_Fouls              50036
A_Fouls              50036
H_Yellow_Cards       43621
A_Yellow_Cards       43621
Game Link                0
dtype: int64

League                   0
Home                     0
Away                     0
INC                     32
Date                     0
Time                     0
H_Score                 32
A_Score                 32
HT_H_Score              32
HT_A_Score              32
WIN                     32
H_BET                 1156
X_BET                 1156
A_BET                 1156
H_Ball_Possession    44450
A_Ball_Possession    44450
H_Goal_Attempts      43537
A_Goal_Attempts      43537
H_Shots_on_Goal      43540
A_Shots_on_Goal      43540
H_Shots_off_Goal     43999
A_Shots_off_Goal     43999
H_Fouls              50036
A_Fouls              50036
H_Yellow_Cards       43621
A_Yellow_Cards       43621
Game Link                0
dtype: int64

In [ ]:
#filling in nulls in yellow cards column

for i, row in dataset.iterrows():
    inc = str(row['INC'])  # convert to string to ensure proper parsing
    home_yellow_count = inc.count('Yellow_Away')
    away_yellow_count = inc.count('Yellow_Home')
    dataset.at[i, 'A_Yellow_Cards'] =  home_yellow_count
    dataset.at[i, 'H_Yellow_Cards'] =  away_yellow_count

In [ ]:
#creating and filling red cards column

dataset["H_Red_Cards"] = 0
dataset["A_Red_Cards"] = 0

for i, row in dataset.iterrows():
    inc = str(row['INC'])  # convert to string to ensure proper parsing
    home_red_count = inc.count('Red_Card_Away')
    away_red_count = inc.count('Yellow_Card_Home')
    dataset.at[i, 'A_Red_Cards'] =  home_red_count
    dataset.at[i, 'H_Red_Cards'] =  away_red_count

In [ ]:
dataset.info()

Seems like there's extra columns in the `La Liga` and `Serie A` DataFrames. Let's remove them:

In [ ]:
#convert data to datetime
data["date"] = pd.to_datetime(data["date"])

In [ ]:
data["opponent_code"] = data["team2"].astype("category").cat.codes
data